In [1]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.spatial.distance import cdist
import math as m
import random
import Bandit as bnd
import ContextBandit as cb

In [ ]:
def KNN_UCB(bandit,Z, theta, phi): ##X:context vector
    X = 
    n = len(X)
    regrets = []
    n_arms = bandit.k #number of arms
    actions = []
    rewards = []
    assert(n >= n_arms)
    for a in range(n_arms): ##play each arm once regardless of context
        reward = bandit.pull(a, X[a])
        rewards.append(reward)
        actions.append(a)
        regrets.append(bandit.regret)
    for t in range(n_arms,n,1):
        distances= cdist(X[:t], [X[t]], 'Euclidean')
        distances = [dist for sublist in distances for dist in sublist]
        k_a = np.zeros(n_arms)
        index = np.zeros(n_arms) ## to store I_t,k
        for a in range(n_arms): ##evaluate k_a for all a
            U = np.zeros(t)
            for k in range(1,t+1, 1): ##calculate uncertainty on [t-1]
                k_least = np.argpartition(distances, k-1)[:k] ## indices of k nearest neighbours
                N = sum(np.array(actions)[k_least] == a) ##N_t,k
                U[k-1] = m.sqrt(theta* m.log(t)/N) + phi(t) * distances[k_least[-1]] if (N!=0) else phi(t) * distances[k_least[-1]]
            k_a[a] = np.argmin(U) + 1
            k_least = np.argpartition(distances, int(k_a[a]-1))[:int(k_a[a])]
            N = sum(np.array(actions)[k_least] == a)
            S = sum(np.array(rewards)[k_least] * (np.array(actions)[k_least] == a))
            f_hat = S/N if (N!=0) else 0
            index[a] = f_hat + m.sqrt(theta* m.log(t)/N) + phi(t) * distances[k_least[-1]] if(N!=0) else f_hat + phi(t) * distances[k_least[-1]]
        arm = np.argmax(index)
        reward = bandit.pull(arm, X[t])
        regrets.append(bandit.regret)
        actions.append(arm)
        rewards.append(reward)
    return actions, rewards, regrets

## case 3
$Z_i~from~ Unif((0,1)^d), \mathbb{E}(Y_i|Z_i=(z_{ij})) =  \prod_{j=1}^d \sin(\theta_0 z_{ij}+\theta_1 *a),  X_i = h(Z_i);
\\ h:R^d \rightarrow R^D$

In [2]:
Z = np.random.uniform(0,1, size = (500,3))